# How to create a trading bot in Python (2) - for beginners, using RSI, TA-lib, Alpaca, Anaconda.

## The trading bot - code only version

If you want, you can find the previous tutorial about starting with Anaconda, Alpaca, Jupyter Notebook, and creating a simple trading bot in 10 lines of code at:

https://superai.pl/the_simple_trading_bot.html

## Install and import dependencies (all the necessary libraries, packages, modules)

In [1]:
#!pip install alpaca-trade-api

In [2]:
#!pip install TA-lib

In [3]:
#!pip install websocket-client

In [4]:
import alpaca_trade_api as tradeapi
import ast
import json
import numpy as np
import talib as ta
import websocket

## Prepare your data for authentication and authenticate your bot at Alpaca

In [5]:
KEY_ID = "PK27UMTIKLJ35Q04UULA"
SECRET_KEY = "agQCs5eDpzsFfkVTi7N5qek6MHI9Qt3pe5Jf0xbt"

In [6]:
api = tradeapi.REST(KEY_ID, SECRET_KEY, "https://paper-api.alpaca.markets", "v2")

## Create variables you might want to change in the future to modify your bot's performance

In [7]:
rsi_timeframe = 3
oversold_threshold = 49
overbought_threshold = 51
company = "AAPL"
shares = 10
data = []

## Create all the necessary functions

In [8]:
def on_open(ws):
    auth_data = {"action": "authenticate", "data": {"key_id": KEY_ID, "secret_key": SECRET_KEY}}
    ws.send(json.dumps(auth_data))
    
    listen_message = {"action":"subscribe","bars":["AAPL"]}
    ws.send(json.dumps(listen_message))
    print("I'm connected to Alpaca API and ready to work. I'm starting to watch the prices.")
    print("I will start calculating RSIs when I'll collect {} prices.".format(rsi_timeframe))
    print("I will buy {} shares of {} when the last RSI is less than {}.".format(shares, company, oversold_threshold))
    print("I will sell {} shares of {} when the last RSI is more than {}.".format(shares, company, overbought_threshold))
    print("So, here we go. Wish me luck.")
    print()

In [9]:
def on_message(ws, message):
    formatted_message = ast.literal_eval(message)
    last_close = formatted_message.get("data").get("c")
    print("Last price after minute closed: {}$".format(last_close))
    
    if isinstance(last_close, float):
        data.append(last_close)
        print('List of last {} collected prices: {}'.format(rsi_timeframe, data[-rsi_timeframe:]))
        
    try:
        if len(data) > rsi_timeframe:
            np_data = np.array(data)
            rsis = ta.RSI(np_data, rsi_timeframe)
            rsi_now = rsis[-1]
            
            print("The list of last 5 RSIs:", rsis[-5:])
            print("Last RSI: ", rsi_now)
            
            if rsi_now < oversold_threshold:
                try:
                    api.get_position(company)
                    print("We hit the threshold to buy, but we already have some shares, so we won't buy more.")
                except:
                    api.submit_order(symbol=company, qty=shares, side = "buy", type='market', time_in_force='gtc')
                    print('We submitted the order to buy {} {} shares.'.format(shares, company))
            
            elif rsi_now > overbought_threshold:
                try:
                    api.get_position(company)
                    api.submit_order(symbol=company,qty=shares,side='sell',type='market',time_in_force='gtc')
                    print('We submitted an order to sell {} {} shares.'.format(shares, company))
                except:
                    print("We hit the threshold to sell, but we don't have anything to sell. Next time maybe.")
            
            else:
                print("The RSI is {} and it's between the given thresholds: {} and {}, so we wait.".format(rsi_now, oversold_threshold, overbought_threshold))
        else:
            print("Not enough prices to calculate RSI and start trading:", len(data), "<=", rsi_timeframe)
    except:
        print("I tried my best, buy I think something went wrong. I'll try again in a moment.")
    print()

## Connect to websocket and ask your bot nicely to start buying and selling stocks for you

In [10]:
#socket = "wss://data.alpaca.markets/stream"
socket = "wss://stream.data.alpaca.markets/v2/sip"
SuperAI_trader = websocket.WebSocketApp(socket, on_open=on_open, on_message=on_message)

In [11]:
SuperAI_trader.run_forever()
print("You've interrupted me. That's it then. I hope I did good. Till the next time.")

I'm connected to Alpaca API and ready to work. I'm starting to watch the prices.
I will start calculating RSIs when I'll collect 3 prices.
I will buy 10 shares of AAPL when the last RSI is less than 49.
I will sell 10 shares of AAPL when the last RSI is more than 51.
So, here we go. Wish me luck.

You've interrupted me. That's it then. I hope I did good. Till the next time.


## Modify the bot

In [12]:
rsi_timeframe = 4
oversold_threshold = 48
overbought_threshold = 52
company = "TSLA"
shares = 100
data = []

In [13]:
SuperAI_trader.run_forever()
#print("You've interrupted me. That's it then. I hope I did good. Till the next time.")

I'm connected to Alpaca API and ready to work. I'm starting to watch the prices.
I will start calculating RSIs when I'll collect 4 prices.
I will buy 100 shares of TSLA when the last RSI is less than 48.
I will sell 100 shares of TSLA when the last RSI is more than 52.
So, here we go. Wish me luck.



True

### Pretty much the same bot in 15 lines of code

In [14]:
import alpaca_trade_api as tradeapi, ast, json, numpy as np, talib as ta, websocket
data = []
def on_open(ws):
    ws.send(json.dumps({"action": "authenticate", "data": {"key_id": "PK3O8OD82FIXEEPGT8GP", "secret_key": "9tCxXfAtHpeUi6qlhdL9i7dGHt6qgDCNzQreBkcZ"}}))
    ws.send(json.dumps({"action": "listen", "data": {"streams": ["AM.AAPL"]}}))
def on_message(ws, message):
    if isinstance(ast.literal_eval(message).get("data").get("c"), float): data.append(ast.literal_eval(message).get("data").get("c"))        
    if len(data) > 3 and ta.RSI(np.array(data), 3)[-1] < 49:
        try: tradeapi.REST("PK3O8OD82FIXEEPGT8GP", "9tCxXfAtHpeUi6qlhdL9i7dGHt6qgDCNzQreBkcZ", "https://paper-api.alpaca.markets", "v2").get_position("AAPL")
        except:tradeapi.REST("PK3O8OD82FIXEEPGT8GP", "9tCxXfAtHpeUi6qlhdL9i7dGHt6qgDCNzQreBkcZ", "https://paper-api.alpaca.markets", "v2").submit_order(symbol="AAPL", qty=10, side = "buy", type='market', time_in_force='gtc')            
    elif len(data) > 3 and ta.RSI(np.array(data), 3)[-1] > 51:
        tradeapi.REST("PK3O8OD82FIXEEPGT8GP", "9tCxXfAtHpeUi6qlhdL9i7dGHt6qgDCNzQreBkcZ", "https://paper-api.alpaca.markets", "v2").get_position("AAPL")
        tradeapi.REST("PK3O8OD82FIXEEPGT8GP", "9tCxXfAtHpeUi6qlhdL9i7dGHt6qgDCNzQreBkcZ", "https://paper-api.alpaca.markets", "v2").submit_order(symbol="AAPL",qty=10,side='sell',type='market',time_in_force='gtc')
websocket.WebSocketApp("wss://data.alpaca.markets/stream", on_open=on_open, on_message=on_message).run_forever()

True